#### Notebook to create script to use PLATZI API

In [44]:
!pip install requests
!pip install pandas


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/28/30/8114832daff7489f179971dbc1d854109b7f4365a546e3ea75b6516cea95/pandas-2.3.2-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for numpy>=1.26.0 from https://files.pythonhosted.org/packages/32/ee/de999f2625b80d043d6d2d628c07d0d5555a677a3cf78fdf868d409b8766/numpy-2.3.3-cp312-cp312-win_amd64.whl.metadata
     ---------------------------------------- 0.0/60.9 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.9 kB ? eta -:--:--
     ------------------------------- ------ 51.2/60.9 kB 871.5 kB/s eta 0:00:01
     ------------------------------- ------ 51.2/60.9 kB 871.5 kB/s eta 0:00:01
     -------------------------------------- 60.9/60.9 kB 359.5 kB/s eta 0:00:00
  Obtaining dependency information for pytz>=2020.1 from https://files.pythonhosted.org/packages/81/c4/34e93fe5f5429d7570ec1fa436f1986fb1f00c3e0f43a589fe2bbcd22c3f/pytz-2025.2-py2.py3-none-any.whl.


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [132]:
import requests
import pandas as pd
import random
from datetime import datetime, timedelta

In [46]:
product_url = "https://api.escuelajs.co/api/v1/products"
user_url = "https://api.escuelajs.co/api/v1/users"

In [47]:
def fetch_data(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return {"error": "Failed to retrieve data"}

In [134]:
ex_product = fetch_data(product_url)
ex_product[0]

{'id': 2,
 'title': 'Classic Red Pullover Hoodie',
 'slug': 'classic-red-pullover-hoodie',
 'price': 10,
 'description': 'Elevate your casual wardrobe with our Classic Red Pullover Hoodie. Crafted with a soft cotton blend for ultimate comfort, this vibrant red hoodie features a kangaroo pocket, adjustable drawstring hood, and ribbed cuffs for a snug fit. The timeless design ensures easy pairing with jeans or joggers for a relaxed yet stylish look, making it a versatile addition to your everyday attire.',
 'category': {'id': 1,
  'name': 'Clothes',
  'slug': 'clothes',
  'image': 'https://i.imgur.com/QkIa5tT.jpeg',
  'creationAt': '2025-09-09T21:45:06.000Z',
  'updatedAt': '2025-09-09T21:45:06.000Z'},
 'images': ['https://i.imgur.com/1twoaDy.jpeg',
  'https://i.imgur.com/FDwQgLy.jpeg',
  'https://i.imgur.com/kg1ZhhH.jpeg'],
 'creationAt': '2025-09-09T21:45:06.000Z',
 'updatedAt': '2025-09-09T21:45:06.000Z'}

In [133]:
ex_user = fetch_data(user_url)
ex_user[0]

{'id': 1,
 'email': 'john@mail.com',
 'password': 'changeme',
 'name': 'Jhon',
 'role': 'customer',
 'avatar': 'https://i.imgur.com/LDOO4Qs.jpg',
 'creationAt': '2025-09-09T21:45:06.000Z',
 'updatedAt': '2025-09-09T21:45:06.000Z'}

### Extract useful information - Product

id should be renamed to product_id, and the id within category should be renamed to category_id as it is likely a foreign key.

Title and slug are redundant, and since I do not plan to display products, slug will be discarded.

Only the first image link will be used, in case I the need to display the product arises.

There are two pairs of created and updated, so one pair will be removed.

In [118]:
ex_product_cleaned = {
    'product_id': ex_product[0]['id'],
    'category_id': ex_product[0]['category']['id'],
    'title': ex_product[0]['title'],
    'price': ex_product[0]['price'],
    'description': ex_product[0]['description'],
    'creationAt': ex_product[0]['creationAt'],
    'updatedAt': ex_product[0]['updatedAt']
}

In [119]:
ex_product_cleaned

{'product_id': 2,
 'category_id': 1,
 'title': 'Classic Red Pullover Hoodie',
 'price': 10,
 'description': 'Elevate your casual wardrobe with our Classic Red Pullover Hoodie. Crafted with a soft cotton blend for ultimate comfort, this vibrant red hoodie features a kangaroo pocket, adjustable drawstring hood, and ribbed cuffs for a snug fit. The timeless design ensures easy pairing with jeans or joggers for a relaxed yet stylish look, making it a versatile addition to your everyday attire.',
 'creationAt': '2025-09-09T21:45:06.000Z',
 'updatedAt': '2025-09-09T21:45:06.000Z'}

In [120]:
product_rows = []
product_images_rows = []

for p in ex_product:
    product_rows.append({
        'product_id': p['id'],
        'category_id': p['category']['id'],
        'title': p['title'],
        'price': p['price'],
        'description': p['description'],
        'creationAt': p['creationAt'],
        'updatedAt': p['updatedAt']
    })
    product_images_rows.append({
        'product_id': p['id'],
        'image': p['category']['image']
    })

df_products = pd.DataFrame(product_rows).drop_duplicates().set_index('product_id')
df_products.head()

,category_id,title,price,description,creationAt,updatedAt
product_id,,,,,,
2,1,Classic Red Pullover Hoodie,10,Elevate your casual wardrobe with our Classic ...,2025-09-09T21:45:06.000Z,2025-09-09T21:45:06.000Z
3,1,Classic Heather Gray Hoodie,69,Stay cozy and stylish with our Classic Heather...,2025-09-09T21:45:06.000Z,2025-09-09T21:45:06.000Z
4,1,Classic Grey Hooded Sweatshirt,90,Elevate your casual wear with our Classic Grey...,2025-09-09T21:45:06.000Z,2025-09-09T21:45:06.000Z
5,1,Classic Black Hooded Sweatshirt,79,Elevate your casual wardrobe with our Classic ...,2025-09-09T21:45:06.000Z,2025-09-09T21:45:06.000Z
6,1,Classic Comfort Fit Joggers,25,Discover the perfect blend of style and comfor...,2025-09-09T21:45:06.000Z,2025-09-09T21:45:06.000Z


In [100]:
df_products_images = pd.DataFrame(product_images_rows).drop_duplicates()
df_products_images.head()

,product_id,image
0,2,https://i.imgur.com/QkIa5tT.jpeg
1,3,https://i.imgur.com/QkIa5tT.jpeg
2,4,https://i.imgur.com/QkIa5tT.jpeg
3,5,https://i.imgur.com/QkIa5tT.jpeg
4,6,https://i.imgur.com/QkIa5tT.jpeg


This makes much more sense, and removes a lot of redundant information. However, it must be noted that within every product was a section for the category that the product was in, which means it should belong to another table as it would be redundant to keep the information of category which is more broad than the product.

This means another table, Category, will be created. 

Also note that there are images for products, as well as images in Category that are repeating groups of image links. It would be useful to separate the images from the other data, so two more tables should be created, one for Product Images, and then another for Category Images.

In [113]:
category_rows = []
category_images_rows = []

for p in ex_product:
    category_rows.append({
        'category_id': p['category']['id'],
        'name': p['category']['name']
    })
    category_images_rows.append({
        'category_id': p['category']['id'],
        'image': p['category']['image']
        })

df_category = pd.DataFrame(category_rows).drop_duplicates().reset_index(drop=True)
df_category

,category_id,name
0,1,Clothes
1,2,Electronics
2,3,Furniture
3,4,Shoes
4,5,Miscellaneous


In [114]:
df_category_images = pd.DataFrame(category_images_rows).drop_duplicates().reset_index(drop=True)
df_category_images

,category_id,image
0,1,https://i.imgur.com/QkIa5tT.jpeg
1,2,https://i.imgur.com/ZANVnHE.jpeg
2,3,https://i.imgur.com/Qphac99.jpeg
3,4,https://i.imgur.com/qNOjJje.jpeg
4,5,https://i.imgur.com/BG8J0Fj.jpg


### Extract useful information - User

User information has a much more simple structure, and simply due to the nature of this project, I will drop password as I have no use for it, as well as the creationAt and updateAt columns since I have no use for time related information for so few entries.

I have also changed 'id' to 'user_id' for more clarity.

In [129]:
cleaned_user = {
'user_id': ex_user[0]['id'],
 'email': ex_user[0]['email'],
 'name':  ex_user[0]['name'],
 'role': ex_user[0]['role'],
 'avatar': ex_user[0]['avatar']
 }
cleaned_user

{'user_id': 1,
 'email': 'john@mail.com',
 'name': 'Jhon',
 'role': 'customer',
 'avatar': 'https://i.imgur.com/LDOO4Qs.jpg'}

In [ ]:
user_rows = []

for u in ex_user:
    user_rows.append({
        'user_id': u['id'],
        'name': u['name'],
        'email': u['email'],
        'role': u['role']
    })


user_df = pd.DataFrame(user_rows).drop_duplicates(subset=['email']).set_index('user_id')
user_df

,name,email,password,role,creationAt,updatedAt
user_id,,,,,,
1,Jhon,john@mail.com,changeme,customer,2025-09-09T21:45:06.000Z,2025-09-09T21:45:06.000Z
2,Maria,maria@mail.com,12345,customer,2025-09-09T21:45:06.000Z,2025-09-09T21:45:06.000Z
3,Admin,admin@mail.com,admin123,admin,2025-09-09T21:45:06.000Z,2025-09-09T21:45:06.000Z
4,Nicolas,nico@gmail.com,1234,customer,2025-09-09T22:11:54.000Z,2025-09-09T23:08:58.000Z
5,Mike Slammer,mikeslammer@gmail.com,123456789,customer,2025-09-09T22:15:42.000Z,2025-09-09T23:09:07.000Z
6,Test User,testuihkk@example.com,123456,customer,2025-09-09T22:22:12.000Z,2025-09-09T23:09:15.000Z
17,edwin,wiwin@mail.com,12345678,customer,2025-09-09T23:42:12.000Z,2025-09-09T23:42:12.000Z


### Simulate Orders

I would like to simulate orders, so I will randomly select users, randomly select items to combine into their order, and randomly choose a date for it as well.

It would be useful to separate orders into Orders, and Order Items which details the items in each order.

##### Orders

* Each order will have an order_id, a user_id, and a total for the order, as well as the date the order was placed.

##### Order Items

* Each order has order items, which can be identified using order_item_id, order_id connecting to the order, and product_id connect to the product.
* Also a quantity, and a price.

In [ ]:
orders = []
order_items = []

order_id_counter = 1
order_item_id_counter = 1

for user_id in user_df.index:
    num_orders = random.randint(1, 5)
    for _ in range(num_orders):
        order_date = datetime.now() - timedelta(days=random.randint(1, 365))
        
        # build order
        orders.append({
            "order_id": order_id_counter,
            "user_id": user_id,
            "order_date": order_date
        })

        # add products to order
        products_sample = df_products.sample()-=






